In [1]:
import ee
ee.Initialize()

In [2]:
from iteru import *
from IPython.display import display
import ipywidgets
import datetime

In [3]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [49]:
aoi = Map.aoi

In [50]:
demvis = {'min':488,'max':1863,'palette':['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

In [51]:
dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')\
           .filter(ee.Filter.bounds(aoi))\
           .select('DSM')\
           .median()\
           .clip(aoi)

In [52]:
#Map.add_layer_widgets(dataset,demvis,'GRED_DEM')

In [53]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")\
     .filter(ee.Filter.bounds(aoi))\
     .filter(ee.Filter.date('2022-01-01','2022-02-01'))\
     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))

In [54]:
S2.size().getInfo()

10

In [55]:
vis_param = get_vis_params(S2)

In [56]:
S2_median = S2.median().clip(aoi)

In [57]:
# Map.add_layer_widgets(S2_median,vis_param,'S2_Median')

In [58]:
ndwi = S2_median.normalizedDifference(['B3', 'B8'])

In [59]:
ndwiMasked = ndwi.updateMask(ndwi.gte(0))

In [60]:
#Map.add_layer_widgets(ndwiMasked,{'min':0,'max':0.5,'palette': ['00FFFF', '0000FF']},'NDWI')

In [61]:
v = ee.Image(1).updateMask(ndwiMasked).reduceToVectors(
           geometry = ndwiMasked.geometry(),
           crs =  ndwiMasked.projection(),
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
)

In [62]:
v

In [63]:
def calc_area(feature):
    area = feature.geometry().area(maxError = 1)
    
    return feature.set({'Area':area})

In [64]:
v = v.map(calc_area)

In [65]:
v

In [66]:
x = v.sort('Area', False).first()

In [68]:
x.geometry().area(maxError = 1).getInfo()

377202209.09674734

In [69]:
v = ee.Feature(v.sort('Area', False).first())

In [70]:
Map.add_layer_widgets(v,{'color':'red'},'Vector')

In [71]:
lake_dem = dataset.clip(v)

In [72]:
x = lake_dem.updateMask(lake_dem.gte(170))

In [73]:
Map.add_layer_widgets(x,demvis,'X')

In [74]:
ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = lake_dem.geometry(), 
                         scale = 10, 
                        maxPixels = 1e10)

In [75]:
water_level = ele.getInfo()['DSM']

In [76]:
water_level

604

In [77]:
import numpy as np

In [78]:
#latlng = ee.Image.pixelLonLat().addBands(lake_dem)
latlng = lake_dem.reduceRegion(reducer=ee.Reducer.toList(), geometry=v.geometry(), maxPixels=1e11, scale=30)

In [96]:
elev_list = latlng.getInfo()['DSM']

In [79]:
elev_values = np.array((ee.Array(latlng.get("DSM")).getInfo()),dtype=np.int64)

In [80]:
elev_values.size

427491

In [81]:
volume = water_level - elev_values[elev_values < water_level]

In [82]:
(volume.sum()*900)/1e9

18.6895017

In [83]:
url = ee.FeatureCollection(v).getDownloadURL('geojson')

In [84]:
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/f471fb9b4de8d4729c69450eb1cb7565-ce036634f5dd90889c4e5ab1a6c2ec6a:getFeatures


In [85]:
url2 = lake_dem.getDownloadURL({'name': 'MyPieceOfDEM', 'crs': 'EPSG:32636', 'scale': 35})

In [86]:
print(url2)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/b93679b5aebef6f784774c029644d1d3-11e663dc39429e209632888f63be3174:getPixels


In [87]:
SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filterBounds(aoi)\
    .filterDate('2022-01-01','2022-02-01')\
    .select(['VV','VH'])

In [88]:
sar = SAR.median().clip(aoi)

In [89]:
Map.add_layer_widgets(sar.select(['VV']),{'min': -20, 'max': -7},'SAR_TEST_1')

In [100]:
x = ee.List([1,2,3,4]).map(lambda num: ee.Number(num).add(ee.Number(5)))

In [109]:
y = x.reduce(ee.Reducer.sum())

In [112]:
ee.Number(y).multiply(ee.Number(10)).getInfo()

300

In [107]:
ee.Number(1e5).getInfo()

100000